In [15]:
import pandas as pd
from preprocess import prepare_df
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function
from sklearn.metrics import r2_score


In [2]:
data = pd.read_csv('../lstm/utils/df_imputed.csv', index_col=0).drop(columns=['date'])

In [16]:
df_sensor_s, df_sensor_t, df_gpp_s, df_gpp_t, df_domain_s, df_domain_t, masks = prepare_df(data)

In [18]:
DEVICE = torch.device("cuda:" + str(1))

In [98]:
# Part 1: feature extraction
class CausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation=1, **kwargs):
        super().__init__()
        self.pad = (kernel_size - 1) * dilation
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=self.pad, dilation=dilation, **kwargs)
    
    def forward(self, x):
        x = self.conv(x)
        return x[:, :, :-self.conv.padding[0]]

class ConvFeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_blocks = nn.Sequential(
            CausalConv1d(11, 32, 2),
            nn.ReLU(),
            CausalConv1d(32, 64, 2),
            nn.ReLU(),
            CausalConv1d(64, 128, 2),
        )
    def forward(self, x):
        return self.conv_blocks(x)

In [201]:
# top branch: regressor 
class RecurrentRegressor(nn.Module):
    # gets a sequence of shape (seq len, batch size, n_channels)
    def __init__(self, input_dim, hidden_dim, num_layers, bidirectional):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if self.bidirectional else 1

        self.rnn = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=bidirectional)
        self.fc = nn.Sequential(
            nn.Linear(self.num_directions * hidden_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        output, (h, c) = self.rnn(x)
        # output: seq_len, batch, num_directions * hidden_size
        T, N, C = output.shape
        output = output.view(T*N, -1) # seq_len * batch, num_directions * hidden_size
        output = self.fc(output).view(T, N, -1) #  seq_len, batch, 1
        return output

In [202]:
# one domain discriminator for every source domain
class DomainClassifier(nn.Module):
    # gets a sequence of shape (seq len, batch size, n_channels) --> (batch_size, seq_len * n_channels)
    def __init__(self, n_channels):
        super().__init__()
        self.n_channels = n_channels
        self.fc = nn.Sequential(
            nn.Linear(n_channels, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        # x: (seq len, batch size, n_channels)
        # average over the seq_len dim
        x = torch.mean(x, dim=1) # x: (batch_size, n_channels)
        y_pred = self.fc(x)
        return y_pred

In [203]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [204]:
class Model(nn.Module):
    def __init__(self, extractor, regressor, classifiers):
        super().__init__()
        self.extractor = extractor
        self.regressor = regressor
        self.classifiers = classifiers

    def forward(self, s, t, alpha):
        s = s.permute(0, 2, 1)
        t = t.permute(0, 2, 1)

        s_features = self.extractor(s)
        t_features = self.extractor(t)
        
        s_features = s_features.permute(2, 0, 1)
        t_features = t_features.permute(2, 0, 1)
        y_pred = self.regressor(s_features)
        
        rev_s_features = ReverseLayerF.apply(s_features, alpha)
        rev_t_features = ReverseLayerF.apply(t_features, alpha)
    
        s_preds = []
        t_preds = []

        for classifier in self.classifiers:
            s_domain_pred = classifier(rev_s_features)
            t_domain_pred = classifier(rev_t_features)
            s_preds.append(s_domain_pred)
            t_preds.append(t_domain_pred)

        return y_pred, s_preds, t_preds

In [205]:
def loss_fn(source_domains, target_domains, y, source_preds, target_preds, y_pred, mu):
    reg_loss = nn.MSELoss()(y_pred, y)
    domain_losses = []
    for i in range(9):
        loss2 = nn.BCELoss()(source_preds[i].squeeze(0), source_domains)
        loss3 = nn.BCELoss()(target_preds[i].squeeze(0), target_domains)
        domain_losses.append(loss2 + loss3)
    domain_loss = mu * torch.mean(torch.tensor(domain_losses))
    loss = reg_loss + domain_loss
    return loss, domain_loss, reg_loss

In [208]:
DOMAINS = 9
feature_extractor = ConvFeatureExtractor()
regressor = RecurrentRegressor(128, 128, 1, False)
classifiers = []
for t in range(DOMAINS):
    classifiers.append(DomainClassifier(128).to(DEVICE))

model = Model(feature_extractor, regressor, classifiers).to(DEVICE)
optimizer = torch.optim.RMSprop(model.parameters())

In [209]:
import pdb
EPOCHS = 1000
alpha = 0.1
for epoch in range(EPOCHS):
    train_loss = 0.0
    train_reg_loss = 0.0
    train_domain_loss = 0.0
    train_r2 = 0.0
    model.train()
    for t in range(len(df_sensor_t)):
        domain_losses = []
        x_t = torch.FloatTensor(df_sensor_t[t].values).unsqueeze(1).to(DEVICE)
        y_t = torch.FloatTensor(df_gpp_t[t].values).to(DEVICE)
        domain_t  = torch.FloatTensor([1]).to(DEVICE)
        for k in range(DOMAINS):
          s_k = np.random.choice(range(len(df_sensor_s[k])))
          x_s  = torch.FloatTensor(df_sensor_s[k][s_k].values).unsqueeze(1).to(DEVICE)
          y_s  = torch.FloatTensor(df_gpp_s[k][s_k].values).to(DEVICE)
          domain_s = torch.FloatTensor([0]).to(DEVICE)

          y_pred, s_preds, t_preds = model(x_s, x_t, alpha)
          y_pred = y_pred.squeeze()
          # Get loss and update
          optimizer.zero_grad()

          loss = loss_fn(domain_s, domain_t, y_s, s_preds, t_preds, y_pred, 1)
          domain_losses.append(loss)
          train_r2 += r2_score(y_true=y_s.detach().cpu(), y_pred=y_pred.detach().cpu())


        worst_domain = torch.tensor(domain_losses).argmin(axis=0)[1]
        loss = domain_losses[worst_domain][0]
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_domain_loss += domain_losses[worst_domain][1].item()
        train_reg_loss += domain_losses[worst_domain][2].item()

    
    train_loss /= len(df_sensor_t)
    train_domain_loss /=  len(df_sensor_t)
    train_reg_loss /= len(df_sensor_t)
    train_r2 /= len(df_sensor_t)*DOMAINS
    print(f"Epoch: {epoch+1}/{EPOCHS}")
    print(f"Train loss: {train_loss:.4f} | Reg loss: {train_reg_loss:.4f} | Domain loss: {train_domain_loss:.4f} | R2: {train_r2:.4f}")
        
        


Epoch: 1/1000
Train loss: 40.4315 | Reg loss: 38.8818 | Domain loss: 1.5497 | R2: -37.7259
Epoch: 2/1000
Train loss: 2.5957 | Reg loss: 0.9997 | Domain loss: 1.5960 | R2: 0.0000
Epoch: 3/1000
Train loss: 2.5960 | Reg loss: 0.9996 | Domain loss: 1.5964 | R2: 0.0000
Epoch: 4/1000
Train loss: 2.5954 | Reg loss: 0.9994 | Domain loss: 1.5961 | R2: 0.0000
Epoch: 5/1000
Train loss: 2.5950 | Reg loss: 0.9996 | Domain loss: 1.5954 | R2: 0.0000
Epoch: 6/1000
Train loss: 2.5977 | Reg loss: 0.9997 | Domain loss: 1.5980 | R2: 0.0000
Epoch: 7/1000
Train loss: 2.5953 | Reg loss: 0.9998 | Domain loss: 1.5955 | R2: 0.0000
Epoch: 8/1000
Train loss: 2.5977 | Reg loss: 0.9996 | Domain loss: 1.5981 | R2: 0.0000
Epoch: 9/1000
Train loss: 2.5945 | Reg loss: 0.9997 | Domain loss: 1.5948 | R2: 0.0000
Epoch: 10/1000
Train loss: 2.5952 | Reg loss: 0.9996 | Domain loss: 1.5956 | R2: 0.0000
Epoch: 11/1000
Train loss: 2.5960 | Reg loss: 0.9996 | Domain loss: 1.5964 | R2: -0.0000
Epoch: 12/1000
Train loss: 2.5963 | 

Epoch: 94/1000
Train loss: 1.8321 | Reg loss: 0.4050 | Domain loss: 1.4271 | R2: 0.3895
Epoch: 95/1000
Train loss: 1.7607 | Reg loss: 0.3284 | Domain loss: 1.4323 | R2: 0.3548
Epoch: 96/1000
Train loss: 1.7667 | Reg loss: 0.3354 | Domain loss: 1.4313 | R2: 0.3364
Epoch: 97/1000
Train loss: 1.7231 | Reg loss: 0.2891 | Domain loss: 1.4340 | R2: 0.3364
Epoch: 98/1000
Train loss: 1.7519 | Reg loss: 0.3205 | Domain loss: 1.4314 | R2: 0.2971
Epoch: 99/1000
Train loss: 1.6858 | Reg loss: 0.2509 | Domain loss: 1.4349 | R2: 0.3059
Epoch: 100/1000
Train loss: 1.6847 | Reg loss: 0.2542 | Domain loss: 1.4305 | R2: 0.2636
Epoch: 101/1000
Train loss: 1.6892 | Reg loss: 0.2579 | Domain loss: 1.4313 | R2: 0.2795
Epoch: 102/1000
Train loss: 1.8563 | Reg loss: 0.4247 | Domain loss: 1.4317 | R2: 0.3758
Epoch: 103/1000
Train loss: 1.7371 | Reg loss: 0.2986 | Domain loss: 1.4386 | R2: 0.3894
Epoch: 104/1000
Train loss: 1.8132 | Reg loss: 0.3749 | Domain loss: 1.4383 | R2: 0.3900


KeyboardInterrupt: 

In [211]:
y_pred, s_preds, t_preds = model(x_s, x_t, alpha)
y_pred = y_pred.squeeze()





In [197]:
x_s.shape

torch.Size([5479, 1, 11])

In [212]:
from plotly import graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=y_s.detach().cpu(),
                    mode='markers',
                    name='gt', 
                    marker=dict(size=3)))

fig.add_trace(go.Scatter(y=y_pred.detach().cpu(),
                    mode='markers',
                    name='pred', 
                    marker=dict(size=3)))

In [180]:
for i in range(len(df_sensor_s)):
    print(len(df_sensor_s[i]))

14
1
3
3
4
5
2
9
1
